In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
selected_features = df.drop(columns=['koi_disposition'])
y = df['koi_disposition']

selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=1)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# label encode dataset
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# convert encoded labels to one-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
# create deep learning model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Neural Network model here
model2 = Sequential()
model2.add(Dense(units=100, activation='relu', input_dim=40))
model2.add(Dense(units=100, activation='relu'))
model2.add(Dense(units=3, activation='softmax'))

In [9]:
# Compile and fit the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
164/164 - 1s - loss: 0.6078 - accuracy: 0.7149
Epoch 2/100
164/164 - 0s - loss: 0.3789 - accuracy: 0.7992
Epoch 3/100
164/164 - 0s - loss: 0.3657 - accuracy: 0.8106
Epoch 4/100
164/164 - 0s - loss: 0.3538 - accuracy: 0.8205
Epoch 5/100
164/164 - 0s - loss: 0.3497 - accuracy: 0.8230
Epoch 6/100
164/164 - 0s - loss: 0.3394 - accuracy: 0.8394
Epoch 7/100
164/164 - 0s - loss: 0.3311 - accuracy: 0.8428
Epoch 8/100
164/164 - 0s - loss: 0.3311 - accuracy: 0.8411
Epoch 9/100
164/164 - 0s - loss: 0.3239 - accuracy: 0.8478
Epoch 10/100
164/164 - 0s - loss: 0.3170 - accuracy: 0.8545
Epoch 11/100
164/164 - 0s - loss: 0.3148 - accuracy: 0.8535
Epoch 12/100
164/164 - 0s - loss: 0.3184 - accuracy: 0.8503
Epoch 13/100
164/164 - 0s - loss: 0.3100 - accuracy: 0.8575
Epoch 14/100
164/164 - 0s - loss: 0.3021 - accuracy: 0.8631
Epoch 15/100
164/164 - 0s - loss: 0.2993 - accuracy: 0.8674
Epoch 16/100
164/164 - 0s - loss: 0.2999 - accuracy: 0.8680
Epoch 17/100
164/164 - 0s - loss: 0.3022 - accura

# Quantify the Model

In [10]:
model_loss, model_accuracy = model2.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2550 - accuracy: 0.8987
Normal Neural Network - Loss: 0.2550371587276459, Accuracy: 0.8987414240837097


# Predict

In [14]:
encoded_predictions = model2.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [15]:
print(f"Predicted classes: {prediction_labels[:10]}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED'
 'FALSE POSITIVE' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CANDIDATE'
 'CANDIDATE' 'FALSE POSITIVE']
Actual Labels: ['CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']
